In [2]:
import numpy as np
from model import VAD
from dataset import VADDataset
from sklearn import metrics
from torch.utils.data import DataLoader
import torch
import os

In [3]:
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   
os.environ["CUDA_VISIBLE_DEVICES"]="4"


device = 'cuda:0'

In [4]:
vad = VAD.load_model('./logs/ckpts/epoch20.pth.tar')
dataset = VADDataset(json_path = './', mode='test', sample_rate = 16000, frame_size = 40)

Processing 1680 of 1680
Done!
Merging frames 130543 of 130543
label_zero : 1370.0, label_one : 129173.0

Augment 127803 zeros to dataset...
Shuffling...
Final frames :  258346
Done!


In [5]:
dataloader = DataLoader(dataset, batch_size=1024)

In [6]:
f1_mean = 0
accuracy = 0
for idx, batch in enumerate(dataloader):
    gt = batch['label']
    b = gt.shape[0]
    with torch.no_grad():
        est = vad(batch['wav'])
        est = est.cpu().numpy()
        est = np.argmax(est, axis=1)
    gt = np.argmax(gt.numpy(), axis=1)
    f1_mean += metrics.f1_score(gt, est) * b
    accuracy += metrics.accuracy_score(gt, est) * b
    #print(metrics.f1_score(gt, est), metrics.accuracy_score(gt, est))


print("f1 score : {}".format(f1_mean / len(dataset)))
print("accuracy : {}".format(accuracy / len(dataset)))

f1 score : 0.7203844823918756
accuracy : 0.7738575398883668
